In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

C:\Users\pli6894\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
#Load data into Pandas dataframe

df = pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
# drop coloumns not needed for neural network
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
v1    5572 non-null object
v2    5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


# **See Data_Engineering.ipynb for data distribution research

In [5]:
# Create input and output vectors
# Process the labels
# LabelEncoder() is used to encode labels with value between 0 and n_classes-1

X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [7]:
#Split data into training & test
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2, stratify=Y) #stratify = Y to represent the disproportion of ham & spam in the data

In [8]:
print(len(X_train))
print(len(X_test))
print(len(Y_train))
print(len(Y_test))
print(Y_test)

4457
1115
4457
1115
[[0]
 [0]
 [1]
 ...
 [0]
 [0]
 [0]]


In [9]:
# Process the data via 1) Tokenize the data and convert the text to sequences; 2) Add padding to ensure that all the sequences have the same shape.
# There are many ways of taking the max_len and here an arbitrary length of 150 is chosen.

max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [19]:
sequences_matrix

array([[  0,   0,   0, ..., 762, 226, 810],
       [  0,   0,   0, ...,   7,  31,  89],
       [  0,   0,   0, ..., 395, 410, 131],
       ...,
       [  0,   0,   0, ..., 145, 249,  20],
       [  0,   0,   0, ..., 211,   4,  25],
       [  0,   0,   0, ..., 209, 822, 167]])

# LSTM

In [11]:
# Define LSTM structure
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [12]:
# Call the function & compile the model

model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [13]:
# Fit on the training data
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10, validation_data=(sequences_matrix,Y_train),
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 4457 samples, validate on 4457 samples
Epoch 1/10
4457/4457 [==============================] - 7s 2ms/step - loss: 0.2917 - acc: 0.8874 - val_loss: 0.1062 - val_acc: 0.9791
Epoch 2/10
4457/4457 [==============================] - 7s 2ms/step - loss: 0.0695 - acc: 0.9814 - val_loss: 0.0448 - val_acc: 0.9863
Epoch 3/10
4457/4457 [==============================] - 8s 2ms/step - loss: 0.0415 - acc: 0.9881 - val_loss: 0.0299 - val_acc: 0.9910
Epoch 4/10
4457/4457 [==============================] - 7s 2ms/step - loss: 0.0313 - acc: 0.9908 - val_loss: 0.0270 - val_acc: 0.9921
Epoch 5/10
4457/4457 [==============================] - 7s 2ms/step - loss: 0.0256 - acc: 0.9924 - val_loss: 0.0163 - val_acc: 0.9960
Epoch 6/10
4457/4457 [==============================] - 7s 2ms/step - loss: 0.0197 - acc: 0.9948 - val_loss: 0.0128 - val_acc: 0.9969
Epoch 7/10
4457/4457 [==============================] - 7s 2ms/step - loss: 0.0152 - acc: 0.9962 - val_loss: 0.0099 - val_acc: 0.9971
Epoch 8/10
445

# The model performs well on the validation set and this configuration is chosen as the final model.

In [52]:
# Process the test data set
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [53]:
# Evaluate model on test set
accr = model.evaluate(test_sequences_matrix,Y_test)

1115/1115 [==============================] - 1s 495us/step


In [54]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.063
  Accuracy: 0.985
